In [ ]:
class Nbdos:

    def __init__(self):
        self.hck = {}

    def list_neighbor(self, data, sfs, xi, k):
        neigh = NearestNeighbors(n_neighbors=k)
        neigh.fit(data.iloc[sfs, :data.shape[1]], data.iloc[sfs, -1])
        return neigh.kneighbors([xi], return_distance=False).tolist()[0]

    def nbdos(self, data, sc, k, k2_neighbors, rTh, nTh):
        sfs = []
        cl = [0] * sc.shape[0]
        #print("aqui foi")
        for indeces, xi in sc.iterrows():
            tem = [a for a in self.list_neighbor(data, k2_neighbors, xi, k) if data.iloc[a, -1] == xi.iloc[-1]]
            #print(indeces)
            #print(len(tem))
            if round(len(tem)/k) >= rTh:
                print("Nem se quer aqui")
                sfs.append(indeces)
                self.hck[xi] = tem
        
        for xi in sfs:
            tem = self.list_neighbor(sfs, xi, cl, k)
            print("passa aqui")
            if xi in self.hck.keys():
                self.hck[xi].append(tem)
            else:
                self.hck[xi] = tem

        #print(list(sfs))
        curld = 0
        for xi in sfs:
            if cl[xi] == 0:
                curld += 1
                self.expand_cluster(sfs, xi, cl, curld) #voltar a partir daqui

        for i in range(curld):
            ci = [x for x in range(len(cl)) if cl[x] == i]
            #ci = [xi for j, xi in sc.iterrows() if cl == i]

            if len(ci) < nTh:
                for j in ci:
                    cl[j] = 0
        
        return cl 
    
    def expand_cluster(self, sfsc, xi, cl, curld):
        sfC = [xi]
        cl[xi] = curld

        # Check if list is empty
        while sfC:
            xj = sfC.__getitem__(-1)
            for xl in self.hck[xj]:
                if xl.cl == 0:
                    xl.cl = curld
                    if xl in sfsc:
                        sfC.append(xi)

            sfC.remove(xj)
        return cl


In [198]:
import operator
import math
from functools import reduce

class Instance:

    def __init__(self, label):
        # tuple key where key is the neighbor and the value is the selection weight
        self.neighbors = {}
        self.nk1 = {}
        self.Fs_Fd = {}
        self.cl = 0
        self.label = label

    def set_fs_fd(self, fs):
        self.Fs_Fd = fs

    def get_fs(self):
        return list(self.Fs_Fd.keys())

    def get_fd(self):
        return list(self.Fs_Fd.values())

    def get_nk1(self):
        return self.nk1.keys()

    def add_neighbor(self, neighbor):
        self.neighbors[neighbor] = 0

    def get_list_neighbor(self):
        return list(self.neighbors.keys())

    def get_neighbor_weight(self, neighbor):
        return self.neighbors[neighbor]

    def get_neighbor_high_weight(self):
        return max(self.neighbors, key=self.neighbors.get)

    def set_selection_weight(self, key, w1):
        self.neighbors[key] = (1 + w1)/math.e

    def ss(self, instance):
        """Step 5.3 article
            :return list of neighbors Fs which the distance is no longer than
            the distance between this instance and the parameter
        """
        threshold = k2nn.euclideanDistance(self, instance, len(instance))
        return list({k: v for k, v in self.Fs_Fd.items() if k2nn.euclideanDistance(self, v, len(v)) > threshold}.keys())

    def dpn(self, pri, ari, ss):
        """
        Used to find the instances set, NPN , which belong to the PN neighborhood xi.P N(xj)
        Step 5.3 article + supplementary material
        :param pri: primary reference instance (xi)
        :param ari: assistant reference instance (xj)
        :param ss: returned set from function ss
        :return: the Npn: the set of instances in a radius distance from xi and xj
        """

        npn = [pri, ari]
        midaux = tuple(map(operator.add, pri, ari))
        mid = tuple(map(lambda x: operator.truediv(x, 2), midaux))
        dist_pri_ari = reduce(operator.add, pri+ari)

        for xl in ss:
            dist_mid_xl = k2nn.euclideanDistance(xl, mid, len(xl))
            if dist_mid_xl <= dist_pri_ari:
                npn.append(xl)

        return npn

    def class_entropy(self, gama_mi, class_set):
        gama_m = class_set.shape[0]
        ret = 0

        for c in class_set:
            ret += (c / gama_mi * math.log10(c / gama_mi)) / math.log10(gama_m)

        return ret

    def selection_weight_formula(self, npn, w1, w2, r1, r2, mc, ma, mi, minorities):
        """formula (2) article
            :param mc class c instances
            :param ma majority class instance
            :param mi minority class instance
            :param minorities minorities classes"""

        gama_ma = ma.shape[0]
        gama_mi = mi.shape[0]
        gama_c = mc.shape[0]

        # Minority classes entropy
        e_mi = self.class_entropy(mi, minorities)

        # Majority class entropy
        e_ma = self.class_entropy(mi, minorities)

        overgeneralization_factor = math.exp(r1 * (gama_mi / gama_c) + r2 * e_mi + w2 * (r1*gama_ma/gama_c + 2 * e_ma))
        difficulty_factor = math.exp(-1 * (gama_c / gama_c + gama_ma + gama_mi))

        return 1 / (overgeneralization_factor + w1 * difficulty_factor)


In [ ]:
import pandas as pd
import math
import random as rd
from sklearn.neighbors import NearestNeighbors
import numpy
import pandas as pd

class SMOM:
    def __init__(self):
        self.outstanding = []
        self.trapped = []

    @staticmethod
    def split_classes(data, minority_class):
        c = data[data.iloc[:, -1] == minority_class]
        not_c = data[data.iloc[:, -1] != minority_class]
        return c, not_c

    @staticmethod
    def nearestK3Instances(xi, sc, k1, k2):
        k3 = max([k1,k2])    

        knn3 = NearestNeighbors(n_neighbors=k3)
        knn3.fit(sc.iloc[:, :sc.shape[1]], sc.iloc[:, -1])
        dist, k3neighbors = knn3.kneighbors([xi], return_distance=True)
        
        k3neighbors = k3neighbors.tolist()[0]
        dist = dist.tolist()[0]

        knn1 = NearestNeighbors(n_neighbors=k1)
        knn1.fit(sc.iloc[:, :sc.shape[1]], sc.iloc[:, -1])
        k1neighbors = knn1.kneighbors([xi], return_distance=False).tolist()[0]

        # k3neighbors = knn.getNeighbors(sc, xi, k3)
        # k1neighbors = knn.getNeighbors(sc, xi, k1)        
        k1th = k1neighbors[0]
        distance = numpy.linalg.norm(xi-k1th)
    
        return k3neighbors, dist, k1th, distance, k3

    def selection_weigth(self, data, outstandings, trapped_instances, w1, w2, r1, r2xi, xi_fs_fd):
        """ Return a dict of weight for each instance
            @:param trapped_instance list of type instance
        """
        knn3 = NearestNeighbors(n_neighbors=k3)
        for instance in trapped_instances:                 
            knn3.fit(data.iloc[:, :data.shape[1]], data.iloc[:, -1])
            dist, neighbors = knn3.kneighbors([instance], return_distance=True)
            dist = dist.tolist()[0]
            neighbors = neighbors.tolist()[0]
            
            for neighbor in range(len(neighbors)):
                neighbor_weight = xineighbor
                if (neighbor in self.outstanding) and (instance in neighbor.get_list_neighbor()):
                    instance.set_selection_weight(neighbor, w1)
                elif (instance in neighbor.get_list_neighbor) and (neighbor_weight != 0):
                    instance.set_selection_weight(neighbor_weight)
                else:
                    smaller_distances = instance.ss(neighbor)
                    npn = instance.dpn(instance, neighbor, smaller_distances)
                    ma_class, mi_class, minorities_class_set = self.get_classes(trapped_instances)
                    ma = self.get_class_set(trapped_instances, ma_class)
                    mi = self.get_class_set(trapped_instances, mi_class)
                    minorities = self.get_class_set(trapped_instances, minorities_class_set)
                    instance.set_selection_weight(instance.selection_weight_formula(npn, w1, w2, r1, r2, trapped_instances, ma, mi, minorities))


    @staticmethod
    def filterOutstanding(sc, cl, minor):
        outstanding = []
        trapped = []
        
        for instanceIndex in range(sc[sc.iloc[:, -1] == 16].shape[0]):
            #print(instanceIndex)
            instance = sc.iloc[instanceIndex]
            if cl[instanceIndex] != 0:
                outstanding.append(instance)
            else:
                trapped.append(instance)

        return outstanding, trapped

    def get_classes(self, data):
        # Get majority class
        ma = data.iloc[:, -1].value_counts().idxmax()

        # Get minority class
        mi = data.iloc[:, -1].value_counts().idxmin()

        # Get minority classes set
        mi_aux = data.iloc[:, -1].value_counts() == data.iloc[:, -1].value_counts().min()
        mi_set = mi_aux[mi_aux == True].keys()

        return ma, mi, mi_set

    def get_class_set(self, data, class_name):
        return data[data.iloc[:, -1] == class_name]

    def generate_synthetic_instances(self, sc, g):
        xj = 0
        si = []
        for times in range(g):
            for i in sc:
                if i in self.trapped:
                    xj = i.get_neighbor_high_weight()
                else:
                    xj = rd.choice(i.get_nk1())

                diff = (xj - i)
                gama = [rd.randrange(0, 2) for i in range(len(xj))]
                new_instance = i + diff.dot(gama)

                si.append(new_instance)

        return si

    @staticmethod
    def get_g_for_each_xi(xi, zeta, sc):
        g, remainder = SMOM.get_floor_remainder(zeta, len(sc))

        if xi == sc[-1]: ##
            return g+remainder
        else:
            return g

    @staticmethod
    def get_floor_remainder(zeta, sc_size):
        if (zeta%sc_size) == 0:
            return zeta/sc_size, 0
        else:
            g_for_each_xi = int(math.floor(zeta/sc_size))
            remainder = zeta%sc_size
            return g_for_each_xi, remainder

    @staticmethod
    def nearEnemy(xi, others, k3, distance, neighbors, neighbors_distance):
        knn3 = NearestNeighbors(n_neighbors=k3)
        knn3.fit(others.iloc[:, :others.shape[1]], others.iloc[:, -1])
        dist, k3neighbors = knn3.kneighbors([xi], return_distance=True)
        
        k3neighbors = k3neighbors.tolist()[0]
        dist = dist.tolist()[0]

        fs = []
        fd = []
        for i in range(len(dist)):
            if dist[i] < distance:
                fs.append(k3neighbors[i])
                fd.append(dist[i])

        fs = numpy.concatenate([fs, neighbors])
        fd = numpy.concatenate([fd, neighbors_distance])
        
        return k3EnemyNeighbor, fs, fd        
        
    @staticmethod
    def k2_neighbors(data, k3_neighbor, k3_enemy, xi, k2, k3):
        frames = numpy.concatenate([k3_neighbor, k3_enemy])
        #print(frames)
        knn = NearestNeighbors(n_neighbors=k2)
        knn.fit(data.iloc[frames, :len(frames)], data.iloc[frames, -1])
        return knn.kneighbors([xi], return_distance=False).tolist()[0]        
        


In [205]:
import pandas as pd

data = pd.read_csv("data/data.csv", header=None)
data = data.iloc[:, 1:]
#min_class = data.iloc[:, -1].value_counts().min()
min_class = 16 #classe com 67 instancias
k1 = 12
k2 = 8
rTh = 5/8
nTh = 10
k = 2
w1 = 0.2
w2 = 0.5
r1 = 1/3
r2 = 0.2

#1. Dividindo dados entre classe minoriataria e outros
sc, others =  SMOM.split_classes(data, min_class) 

xi_fs_fd = {}
#2.
for index, xi in sc.iterrows():
    fs_fd = {}
# 2.1)
    k3neighbors, neighbors_distance, k1th, distance, k3 = SMOM.nearestK3Instances(xi, sc, k1, k2)
# 2.2)
    k3EnemyNeighbor, fs, fd = SMOM.nearEnemy(xi, others, k3, distance, k3neighbors, neighbors_distance)
# 2.3)
    k2_neighbors = SMOM.k2_neighbors(data, k3neighbors, k3EnemyNeighbor, xi, k2, k3)
    
    fs_fd[fs] = fd
    xi_fs_fd.append(fs_fd)
#3.
cl = Nbdos().nbdos(data, sc, k2, k2_neighbors, rTh, nTh)
#4.
outstandings, trappeds = SMOM.filterOutstanding(sc, cl, min_class)       
#5.
print(trappeds)
#  5.1)
SMOM().selection_weigth(data, outstandings, trappeds, w1, w2, r1, r2, xi_fs_fd)
#  5.2)
    
#  5.3)

#6.
#  6.1)
#  6.2)

#7.

#8.
#   8.1.
#   8.2
#   8.3.






[1     0.5450
2     0.4250
3     0.1250
4     0.7680
5     0.2940
6     0.1495
7     0.2600
8    16.0000
Name: 7, dtype: float64, 1     0.5400
2     0.4750
3     0.1550
4     1.2170
5     0.5305
6     0.3075
7     0.3400
8    16.0000
Name: 36, dtype: float64, 1     0.5200
2     0.4250
3     0.1650
4     0.9885
5     0.3960
6     0.2250
7     0.3200
8    16.0000
Name: 82, dtype: float64, 1     0.6500
2     0.5200
3     0.1900
4     1.3445
5     0.5190
6     0.3060
7     0.4465
8    16.0000
Name: 142, dtype: float64, 1     0.6450
2     0.4850
3     0.2150
4     1.5140
5     0.5460
6     0.2615
7     0.6350
8    16.0000
Name: 157, dtype: float64, 1     0.6500
2     0.5450
3     0.2300
4     1.7520
5     0.5605
6     0.2895
7     0.8150
8    16.0000
Name: 164, dtype: float64, 1     0.6400
2     0.5000
3     0.1850
4     1.3035
5     0.4445
6     0.2635
7     0.4650
8    16.0000
Name: 197, dtype: float64, 1     0.5900
2     0.5000
3     0.2000
4     1.1870
5     0.4120
6     0.2705
7     0.

TypeError: selection_weigth() takes 6 positional arguments but 7 were given

In [ ]:
frames = numpy.array([0,   54,   51,   43,   44,   62,   25,   21,   36,   64,   61,   32,  222, 3222,\
  2279,  455,   87,  715,  704, 3240, 3809, 2359, 2292,   98,    0,   54,   51,   43])
# knn = NearestNeighbors(n_neighbors=k2)
# knn.fit(data.iloc[frames, :frames.shape[1]], data.iloc[frames, -1])
# knn.kneighbors([xi], return_distance=False).tolist()[0]

#data.iloc[frames, :frames.shape[1]]


In [ ]:
for i in range(sc[sc.iloc[:, -1] == 16].shape[0]):
    print(i)
    